In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gspread


gc = gspread.service_account(filename='/content/drive/MyDrive/py-to-gc-05cd599c852f.json') # 사용할 계정의 정보를 오픈

sh = gc.open("dart2gs") # 저장할 구글 시트를 오픈

#print(sh.sheet1.get('A1'))

[]


In [ ]:
import requests
import json
import pandas as pd

KEY = '발급받은 API 키'

url = 'https://opendart.fss.or.kr/api/fnlttSinglAcnt.json'
params ={
    'crtfc_key' : KEY, # api 인증키
    'corp_code' : '00372226', # 회사 고유번호(삼성전자)
    'bsns_year' : '2022', # 사업연도
    'reprt_code' : '11011' # 보고서 코드; 1분기보고서: 11013, 반기보고서: 11012, 3분기보고서: 11014, 사업보고서: 11011
    }

response = requests.get(url, params=params)

json_data = response.json()

detail = json_data['list']



In [ ]:

# dataframe 구조 확인
#df.head()

,rcept_no,reprt_code,bsns_year,corp_code,stock_code,fs_div,fs_nm,sj_div,sj_nm,account_nm,...,thstrm_dt,thstrm_amount,frmtrm_nm,frmtrm_dt,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_dt,bfefrmtrm_amount,ord,currency
0,20230316001324,11011,2022,00372226,131290,CFS,연결재무제표,BS,재무상태표,유동자산,...,2022.12.31 현재,228247078200,제 27 기,2021.12.31 현재,192217152225,제 27 기초,2021.01.01 현재,147353750354,1,KRW
1,20230316001324,11011,2022,00372226,131290,CFS,연결재무제표,BS,재무상태표,비유동자산,...,2022.12.31 현재,207588760554,제 27 기,2021.12.31 현재,179572183591,제 27 기초,2021.01.01 현재,153719675327,3,KRW
2,20230316001324,11011,2022,00372226,131290,CFS,연결재무제표,BS,재무상태표,자산총계,...,2022.12.31 현재,435835838754,제 27 기,2021.12.31 현재,371789335816,제 27 기초,2021.01.01 현재,301073425681,5,KRW
3,20230316001324,11011,2022,00372226,131290,CFS,연결재무제표,BS,재무상태표,유동부채,...,2022.12.31 현재,71583964725,제 27 기,2021.12.31 현재,75619756227,제 27 기초,2021.01.01 현재,60377853149,7,KRW
4,20230316001324,11011,2022,00372226,131290,CFS,연결재무제표,BS,재무상태표,비유동부채,...,2022.12.31 현재,33476149489,제 27 기,2021.12.31 현재,25990879603,제 27 기초,2021.01.01 현재,15778108941,9,KRW


In [ ]:
df = pd.json_normalize(detail)
df = df.replace('-','0') # 결측치 0으로 변환
df['thstrm_amount'] = df['thstrm_amount'].str.replace(',', '').astype('int64') # 쉼표 구분 제거 및 형변환
df['frmtrm_amount'] = df['frmtrm_amount'].str.replace(',', '').astype('int64')
df['bfefrmtrm_amount'] = df['bfefrmtrm_amount'].str.replace(',', '').astype('int64')
print(df)
worksheet = sh.get_worksheet(0)
worksheet.update([df.columns.values.tolist()] + df.values.tolist()) # 구글 sheet로 업데이트

          rcept_no reprt_code bsns_year corp_code stock_code fs_div   fs_nm  \
0   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
1   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
2   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
3   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
4   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
5   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
6   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
7   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
8   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
9   20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
10  20230316001324      11011      2022  00372226     131290    CFS  연결재무제표   
11  20230316001324      11011      2022  00372226   

/usr/local/lib/python3.10/dist-packages/gspread/worksheet.py:1069: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '1CKUA-Fo4OwwxaHXCYDo6DRvPX2KwMrY9BGyxGGVng7A',
 'updatedRange': "'시트1'!A1:U27",
 'updatedRows': 27,
 'updatedColumns': 21,
 'updatedCells': 567}